In [ ]:
#uncomment this line if you get import errors
!pip install tensorflowjs

In [ ]:
import glob
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import tensorflow
from tensorflow import keras
from keras import models, layers
import tensorflowjs as tfjs
import zipfile
from zipfile import ZipFile

The following three code cells involve loading the images, converting them to the Lab and greyscale colorspaces, and dividing them into testing and training feature/label sets.

In [ ]:
#return the set of grayscale and color images from the given filepath
def getData(filepath):
  rgb_list = []
  gray_list = []
  file_path = glob.glob(os.path.join(filepath,'*.jpg'))

  for img_path in file_path:
    rgb_img = cv2.imread(img_path) #take in the BGR image in the form of a numpy array
    gray_img = cv2.cvtColor(rgb_img, cv2.COLOR_BGR2GRAY) #convert to grayscale using cv2

    rgb_img = cv2.cvtColor(rgb_img, cv2.COLOR_BGR2LAB)
    rgb_img = cv2.resize(rgb_img,(224,224))
    gray_img = cv2.resize(gray_img,(224,224))
    rgb_list.append(rgb_img)
    gray_list.append(gray_img)

  rgb_images = np.stack(rgb_list,axis=0)
  gray_images = np.stack(gray_list,axis=0)
  gray_images = np.reshape(gray_images,(len(gray_images),224,224,1))

  #print(np.shape(rgb_images),np.shape(gray_images))
  return rgb_images , gray_images

#combine L and ab channels into a single image
def combineLab(L, ab):
  outLab = np.empty((224, 224, 3))
  outLab[:, :, 1:3] = ab
  outLab[:, :, 0] = L.reshape(224, 224)
  #outLab[:, :, 0] = np.full((224, 224), .5)
  outLab = (outLab * 255).astype(np.uint8);
  return cv2.cvtColor(outLab, cv2.COLOR_LAB2RGB)

#resize image
def resize(filepath):
  in_path = glob.glob(os.path.join(filepath,'*.jpg'))
  for i in range(0, len(in_path)):
    rgb_img = cv2.imread(in_path[i]) #take in the BGR image in the form of a numpy array
    rgb_img = cv2.resize(rgb_img,(224,224))
    cv2.imwrite(in_path[i], rgb_img)

#resize("drive/MyDrive/ML/landscapes_some")

In [ ]:
with ZipFile('landscapes2000.zip', 'r') as zipObj:
  zipObj.extractall('')

In [ ]:
#prepare features and labels (inputs and outputs, respectively)
training_labels, training_features = getData("training")
training_labels = training_labels / 255.
training_features = training_features / 255.;

testing_labels, testing_features = getData("testing")
testing_labels = testing_labels / 255.
testing_features = testing_features / 255.;

#chop the L component off the Lab image to leave just the ab components
training_labels = training_labels[:, :, :, 1:3]
testing_labels = testing_labels[:, :, :, 1:3]

training_labels.shape

The following cells set up the model, train it, evaluate it using testing data, and show a sample with one image in the testing data.

In [ ]:
#create the model used for colorizing images
input_layer = layers.Input(shape=(224,224,1))
encoder_layer = layers.Conv2D(32, (3, 3), activation = 'relu', padding = 'same')(input_layer)
encoder_layer = layers.MaxPooling2D((2, 2), padding='same')(encoder_layer)
encoder_layer = layers.Conv2D(64, (3, 3), activation = 'relu', padding = 'same')(encoder_layer)
encoder_layer = layers.MaxPooling2D((2, 2), padding='same')(encoder_layer)
encoder_layer = layers.Conv2D(128, (3, 3), activation = 'relu', padding = 'same')(encoder_layer)
encoder_layer = layers.MaxPooling2D((2, 2), padding='same')(encoder_layer)
encoder_layer = layers.Conv2D(256, (3, 3), activation = 'relu', padding = 'same')(encoder_layer)
encoder_layer = layers.MaxPooling2D((2, 2), padding='same')(encoder_layer)

decoder_layer = layers.UpSampling2D((2, 2))(encoder_layer)
decoder_layer = layers.Conv2D(128, (3, 3), activation = 'relu', padding = 'same')(decoder_layer)
decoder_layer = layers.UpSampling2D((2, 2))(decoder_layer)
decoder_layer = layers.Conv2D(64, (3, 3), activation = 'relu', padding = 'same')(decoder_layer)
decoder_layer = layers.UpSampling2D((2, 2))(decoder_layer)
decoder_layer = layers.Conv2D(32, (3, 3), activation = 'relu', padding = 'same')(decoder_layer)
decoder_layer = layers.UpSampling2D((2, 2))(decoder_layer)
decoder_layer = layers.Conv2D(16, (3, 3), activation = 'relu', padding = 'same')(decoder_layer)
output_layer = layers.Conv2D(2, (3, 3), padding = 'same', activation = 'sigmoid')(decoder_layer)

model = models.Model(input_layer, output_layer)
model.compile(optimizer = 'rmsprop', loss = 'mean_squared_error')
model.summary()

In [ ]:
#train and save the model.
#If desired, the top two lines can be commented out and the 3rd line 
#uncommented to load an existing model, as opposed to training a new model.
model.fit(training_features, training_labels, batch_size=32, epochs=10)
model.save('landscapes.keras')
tfjs.converters.save_keras_model(model, 'landscapes.json') #btw, I added this line for exporting the model to js. Comment this out if you're loading an existing model
#model = models.load_model('landscapes35.keras')


In [ ]:
#evaluate the model with both the training and testing data
model.evaluate(training_features, training_labels)
model.evaluate(testing_features, testing_labels)

In [ ]:
#test out the model with a specific image (at index n)
n = 28
greyscale = testing_features[n]
out = model.predict(testing_features[n].reshape(1, 224, 224, 1))
original = testing_labels[n]

plt.figure(figsize=(12, 4), dpi=100)
plt.subplot(1, 3, 1)
plt.imshow(greyscale.reshape(224, 224), cmap='gray')
plt.subplot(1, 3, 2)

plt.imshow(combineLab(greyscale, out))
plt.subplot(1, 3, 3)
plt.imshow(combineLab(greyscale, original))